In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import catboost

*  # Загрузка таблиц

In [2]:
post = pd.read_csv('//kaggle/input/dataset-limrr/post.csv')
user = pd.read_csv('/kaggle/input/dataset-limrr/user.csv')
action = pd.read_csv('/kaggle/input/dataset-limrr/action_lim1kk.csv')
df_2 = pd.read_csv('/kaggle/input/dataset-limrr/df_tar_40000.csv')
df_3 = pd.read_csv('/kaggle/input/dataset-limrr/us_action_2000.csv', sep=';')
df_4 = pd.read_csv('/kaggle/input/dataset-limrr/us_tar_12_24.csv', sep=';')

In [3]:
df_4.head()

,timestamp,user_id,post_id,action,target
0,2021-12-28 22:08:12,143646,3693,view,1
1,2021-12-28 22:11:32,143646,1032,view,1
2,2021-12-28 22:13:38,143646,805,view,1
3,2021-12-28 22:38:45,143646,3718,view,1
4,2021-12-24 16:48:54,137259,1600,view,1


In [4]:
user.head()

,Unnamed: 0,user_id,gender,age,country,city,exp_group,os,source
0,0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,1,201,0,37,Russia,Abakan,0,Android,ads
2,2,202,1,17,Russia,Smolensk,4,Android,ads
3,3,203,0,18,Russia,Moscow,1,iOS,ads
4,4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [5]:
post.head()

,Unnamed: 0,post_id,text,topic
0,0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,2,3,Asian quake hits European shares\n\nShares in ...,business
3,3,4,India power shares jump on debut\n\nShares in ...,business
4,4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [6]:
action.head()

,Unnamed: 0,timestamp,user_id,post_id,action,target
0,0,2021-10-26 14:19:54,11854,4589,view,0
1,1,2021-10-26 14:20:08,11854,6185,view,0
2,2,2021-10-26 14:22:42,11854,3694,view,0
3,3,2021-10-26 14:23:41,11854,3732,view,0
4,4,2021-10-26 14:24:06,11854,7104,view,0


In [7]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  163205 non-null  int64 
 1   user_id     163205 non-null  int64 
 2   gender      163205 non-null  int64 
 3   age         163205 non-null  int64 
 4   country     163205 non-null  object
 5   city        163205 non-null  object
 6   exp_group   163205 non-null  int64 
 7   os          163205 non-null  object
 8   source      163205 non-null  object
dtypes: int64(5), object(4)
memory usage: 11.2+ MB


In [8]:
post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  7023 non-null   int64 
 1   post_id     7023 non-null   int64 
 2   text        7023 non-null   object
 3   topic       7023 non-null   object
dtypes: int64(2), object(2)
memory usage: 219.6+ KB


In [9]:
action.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1000000 non-null  int64 
 1   timestamp   1000000 non-null  object
 2   user_id     1000000 non-null  int64 
 3   post_id     1000000 non-null  int64 
 4   action      1000000 non-null  object
 5   target      1000000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 45.8+ MB


In [10]:
post = post.drop(['Unnamed: 0'], axis=1)
user = user.drop(['Unnamed: 0'], axis=1)
action = action.drop(['Unnamed: 0'], axis=1)


In [11]:
data = pd.concat([action, df_2])
data = pd.concat([data, df_3])
data = pd.concat([data, df_4])

In [12]:
data

,timestamp,user_id,post_id,action,target
0,2021-10-26 14:19:54,11854,4589,view,0
1,2021-10-26 14:20:08,11854,6185,view,0
2,2021-10-26 14:22:42,11854,3694,view,0
3,2021-10-26 14:23:41,11854,3732,view,0
4,2021-10-26 14:24:06,11854,7104,view,0
...,...,...,...,...,...
632030,2021-12-28 21:59:38,143645,2521,view,1
632031,2021-12-28 22:09:47,143645,3631,view,1
632032,2021-12-28 22:13:53,143645,4919,view,1
632033,2021-12-28 22:18:44,143645,1373,view,1


* # Добавляем новые фичи для таблиц Post и User
# 

In [13]:
#Tfidf для текстов постов и длину текста
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True)
tfidf = tfidf_vectorizer.fit_transform(post['text'])
tfidf_dc = tfidf.todense()
post['tfidf_max'] = [np.max(i) for i in tfidf_dc]

post['word_count'] = post['text'].apply(lambda x: len(str(x).split()))

#Для таблицы пользователей средний возраст по городам, количество пользователей в каждой стране и новую категориальную фичу для обозначения группы по возрасту
user['mean_age_per_city'] = user['city'].map(user.groupby('city')['age'].mean())
user['users_in_country'] = user['country'].map(user['country'].value_counts())
bins = [0, 12, 17, 24, 34, 44, 54, 64, float('inf')]  # Определение границ бинов
labels = ['Дети','Подростки', 'Молодежь', 'Молодые взрослые', 'Взрослые', 'Средний возраст', 'Старшее поколение', 'Пожилые']  # Названия групп
user['age_group'] = pd.cut(user['age'], bins=bins, labels=labels, right=False)


In [14]:
#Смерджим таблицу активности пользователей с другими таблицами по айди постов или пользователей
df2 = data.copy()
df2 = pd.merge(df2, post, on = 'post_id' )
df2 = pd.merge(df2, user, on = 'user_id' )


#Поменяем формат колонки timestamp для последующих трансформаций
df2['timestamp'] = df2['timestamp'].astype("datetime64[ns]")
df2['month'] = df2['timestamp'].dt.month
df2['day_of_week'] = df2['timestamp'].dt.day_of_week
df2['hour'] = df2['timestamp'].dt.hour
df2 = df2.sort_values(['timestamp'])
df2 = df2.loc[(df2['action'] == 'view')]

#Разобъем датасет на обучающую и тестовою таблицы 
df_train_1 = df2[:df2.shape[0]*8//10]
df_test_1 = df2[df2.shape[0]*8//10:]

y_train_1 = df_train_1['target']
y_test_1 = df_test_1['target']


#Дропнем лишние колонки и заведем список категориальных колонок
df_train_1 = df_train_1.drop(['target', 'text', 'timestamp', 'action', 'user_id', 'age','post_id'], axis=1)
df_test_1 = df_test_1.drop(['target', 'text', 'timestamp', 'action', 'user_id', 'age','post_id'], axis=1)
cat_col = ['topic', 'gender', 'country', 'city', 'exp_group', 'os', 'source', 'age_group', 'day_of_week', 'hour']

In [15]:
df_train_1.head()

,topic,tfidf_max,word_count,gender,country,city,exp_group,os,source,mean_age_per_city,users_in_country,age_group,month,day_of_week,hour
1433584,sport,0.154628,592,0,Ukraine,Kyiv,3,Android,ads,27.065896,8273,Молодежь,10,4,6
1102596,politics,0.180329,604,1,Ukraine,Dnipro,3,iOS,ads,26.967273,8273,Молодежь,10,4,6
1405223,tech,0.190373,438,1,Russia,Shagonar,0,Android,ads,34.500000,143035,Взрослые,10,4,6
1433529,movie,0.274343,125,0,Ukraine,Kyiv,3,Android,ads,27.065896,8273,Молодежь,10,4,6
1102703,politics,0.154061,324,1,Ukraine,Dnipro,3,iOS,ads,26.967273,8273,Молодежь,10,4,6


In [16]:
clf = catboost.CatBoostClassifier()
clf.fit(df_train_1, y_train_1,
       cat_features=cat_col)

Learning rate set to 0.26488
0:	learn: 0.5540393	total: 2.48s	remaining: 41m 23s
1:	learn: 0.4754948	total: 4.95s	remaining: 41m 8s
2:	learn: 0.4395971	total: 6.89s	remaining: 38m 11s
3:	learn: 0.4210709	total: 8.57s	remaining: 35m 34s
4:	learn: 0.3887760	total: 10.9s	remaining: 36m 16s
5:	learn: 0.3769296	total: 12.5s	remaining: 34m 37s
6:	learn: 0.3699829	total: 14.3s	remaining: 33m 54s
7:	learn: 0.3627641	total: 16.5s	remaining: 34m 4s
8:	learn: 0.3577835	total: 18.6s	remaining: 34m 6s
9:	learn: 0.3563136	total: 21.4s	remaining: 35m 15s
10:	learn: 0.3551733	total: 24.1s	remaining: 36m 6s
11:	learn: 0.3544720	total: 26.9s	remaining: 36m 57s
12:	learn: 0.3529952	total: 29.2s	remaining: 36m 57s
13:	learn: 0.3519277	total: 32s	remaining: 37m 33s
14:	learn: 0.3510061	total: 34.1s	remaining: 37m 16s
15:	learn: 0.3505460	total: 36.1s	remaining: 36m 58s
16:	learn: 0.3499202	total: 37.7s	remaining: 36m 20s
17:	learn: 0.3493602	total: 39.4s	remaining: 35m 46s
18:	learn: 0.3490663	total: 41.3s

In [17]:
clf.save_model('catboost_model')